# Final Strategy for RR Graphs

This file contains the code we run to find our final strategy.
We find the optimal coefficients of a linear combination of eigenvector
centrality, closeness centrality, and triangle centrality. The objective
function is the performance of the linear combination on each graph 
of a certain type (e.g., SNAP) against the best performing teams of the
previous 2-3 days. 

In [29]:
import numpy as np
import networkx as nx
import json
from collections import Counter
import random
import sim
import operator
from heapq import nlargest

In [30]:
# Global Variables
n = 10 # Number of seed nodes

In [31]:
def get_graphs(filenames):
    ''' 
    Function to create graphs and adjacency lists
    given a list of filenames 
    '''
    filepath = '/Users/velis.christ/Downloads/'
    graphs = []
    adj_lists = []
    for filename in filenames:
        with open(filepath+filename+".json") as f:
            adj_list = json.load(f)
        # Create a graph
        G = nx.Graph(adj_list)
        graphs.append(G)
        adj_lists.append(adj_list)
    return graphs, adj_lists

In [32]:
def retrieve_strat_filenames(graph_filename, team_names):
    '''
    Retrieve the filenames of the files containing the teams'
    strategies given a graph filename and a list of teams
    '''
    res = []
    for name in team_names:
        strat_filename = graph_filename + "-" + name + ".json"
        res.append(strat_filename)
    return res

def get_strategies(filenames):
    '''
    Create a list of all the strategies for each of the graph filenames
    given as input
     '''
    filepath = '/Users/velis.christ/Downloads/'
    strategies = []
    for filename in filenames:
        with open(filepath+filename) as f:
            strategy = json.load(f)
        strategies.append(strategy)
    return strategies

In [33]:
def compare_all(adj_list, best_nodes, all_test_nodes):
    '''
    Function to run simulations on a given graph. The function runs a
    simulation of the best_nodes against all seed nodes in the list
    all_test_nodes. This function is used to compare the strategy of a linear
    combination of centrality measures against all strategies from previous
    days on a given graph.
    Input:
    - adj_list: adjacency list of graph to run the simulation on
    - best_nodes: seed nodes of the strategy being tested
    - all_test_nodes: list of seed nodes to simulate against
    Output:
    - results: list of all ouputs of simulation (sim.run)
    '''
    results = []
    # print(all_test_nodes)
    for test_nodes in all_test_nodes:
        node_mappings = dict()
        node_mappings["Best"] = best_nodes
        node_mappings["Test"] = test_nodes
        # print(node_mappings)
        result = sim.run(adj_list, node_mappings)
        results.append(result)
    return results

In [34]:
def best_strategy(graphs, adj_lists, strategies):
    ''' 
    Find the best strategy (linear combination of centrality measures)
    given a set of graphs and competing strategies.
    The graphs are of the same type (e.g., SNAP, Preferential Attachment)
    
    Inputs:
    - graphs: list of nx.Graph()
    - adj_lists: list of dict() representing adjacency lists
    - strategies: list of lists of lists of seed nodes of the strategies

    Output:
    - num_beats: list of the number of Wins on each graph
    - scores: list of the number of nodes conquered on each graph
    best_idx: Index of the best coefficients
    coeffs: The optimal coefficients for the linear combination
    '''

    # Compute centralities for each graph
    eig_cent = []
    close_cent = []
    triangles = []
    for i in range(len(graphs)):
        G = graphs[i]
        eig_cent.append(nx.eigenvector_centrality(G))
        close_cent.append(nx.closeness_centrality(G))
        triangles.append(nx.triangles(G))

    # Find best coefficients by trying many different coefficients
    coeffs = []
    scores = []
    num_beats = []
    for alpha in np.linspace(0,1,num=10):
        for beta in np.linspace(0,1,num=10):
            coeffs.append((alpha, beta))
            our_node_mapping = {}
            score = 0
            num_beat = 0
            # Iterate over each graph to compute the score of the current strategy
            for i in range(len(graphs)):
                G = graphs[i]
                curr_eig_cent = eig_cent[i]
                curr_close_cent = close_cent[i]
                curr_triangles = triangles[i]
                # Compute strategy's centrality scores for each node
                for node in G.nodes():
                    our_node_mapping[node] = alpha*curr_eig_cent[node]+beta*curr_close_cent[node]+(1-alpha-beta)*curr_triangles[node]
                seeds = nlargest(n, our_node_mapping.items(), key=operator.itemgetter(1))
                nodes = [j[0] for j in seeds]
                results = compare_all(adj_lists[i], nodes, strategies[i])
                for res in results:
                    score += res["Best"]
                    num_beat += int(res["Best"] > res["Test"])
            scores.append(score)
            num_beats.append(num_beat)
    max_beats = np.max(num_beats)
    print(max_beats)
    most_beats = [(scores[i], i) for i in range(len(num_beats)) if num_beats[i] >= max_beats]
    best = max(most_beats, key=lambda x: x[0])
    best_idx = best[1]
    return num_beats, scores, best_idx, coeffs[best_idx]

In [35]:
# This is the main code that needs to be run to generate the final strategy
# What needs to be defined:
# - team_names: the names of the competing teams to be used in the optimization
# - Change the graph_filenames used in the for loop to be the graph filenames
#   for the graph you want to optimize for

# team_names = ["LMNOP", "ModeloTime", "NetworkX", "_EAA", "_eek", "alley5", "pareto", \
#     "Charlemagne", "NoSleepAtAll", "_NKS", "_pe2", "LongTailedBeavers", "covid_23"]

# team_names = ["_EAA", "ModeloTime", "NoSleepAtAll", "alley5"]

team_names = ["LMNOP", "NoSleepAtAll", "alley5"]

# Graph filenames
er_graph_filenames = ["RR.10.12", "RR.10.13", "RR.10.14"]
pa_graph_filenames = ["RR.10.23", "RR.10.24"]
ssbm_graph_filenames = ["RR.10.32", "RR.10.33", "RR.10.34"]
caltech_graph_filenames = ["RR.10.42", "RR.20.43", "RR.15.44"]
snap_graph_filenames = ["RR.10.52", "RR.10.53", "RR.10.54"]

# Strategies
strategies = []
for graph_filename in er_graph_filenames:
    er_strategies = []
    strategy_filenames = retrieve_strat_filenames(graph_filename, team_names)
    strats = get_strategies(strategy_filenames)
    for strat in strats:
        er_strategies.append(list(strat.values())[0][0][:10])
    strategies.append(er_strategies)
graphs, adj_lists = get_graphs(er_graph_filenames)
num_beats, scores, best_idx, best_coeffs = best_strategy(graphs, adj_lists, strategies)

4


In [36]:
# Print the output of the best strategy function. The important output is:
# - best_coeffs: coefficients of best linear combination

print(num_beats)
print(scores)
print(best_idx)
print(best_coeffs)
print(scores[best_idx])
print(num_beats[best_idx])

[0, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[1136, 1927, 1927, 1927, 1927, 1927, 1927, 1927, 1927, 1803, 2054, 2047, 1930, 1930, 1930, 1930, 1930, 1902, 2064, 2848, 2054, 2047, 2047, 1930, 1930, 1930, 1930, 3156, 2848, 2848, 2054, 2054, 2047, 2047, 2047, 1930, 3021, 2848, 2848, 2848, 2054, 2054, 2047, 2047, 2047, 3021, 2806, 2848, 2848, 2848, 2054, 2054, 2054, 2047, 3071, 2806, 2806, 2806, 2806, 2848, 2054, 2054, 2054, 3071, 2815, 2815, 2806, 2806, 2806, 2806, 2054, 2054, 3071, 2815, 2815, 2815, 2806, 2806, 2806, 2806, 2054, 2745, 2899, 2815, 2815, 2815, 2815, 2806, 2806, 2806, 2745, 2752, 2752, 2899, 2815, 2815, 2815, 2815, 2806, 2806]
27
(0.2222222222222222, 0.7777777777777777)
3156
4
